In [ ]:
from datetime import datetime
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 40)
import numpy as np
import itertools
import pickle
import json
import yaml
import setup as st
import fx_lib as fx
with open(r'../inp/character_inputs.yaml') as file:
    target_stats = yaml.load(file, Loader=yaml.FullLoader)
with open('../inp/master_data.json') as json_file:
    data = json.load(json_file)

In [ ]:
df_items = pd.read_pickle('../outp/equip_potential.pkl')
df_equipped = df_items[~(df_items.hero == '')].copy()
df_hero_stats, char = fx.hero_json_to_df(df_equipped.hero.unique(), data)
df_hero_stats['hero'] = df_hero_stats['Name']
for j in range(0,6):
    temp_df = df_hero_stats.merge(df_equipped[['hero','id']][df_equipped.Type == j], \
                        how = 'left', on = 'hero' )
    df_hero_stats[str(j)] = temp_df['id']

#### Look at equipment for a specific hero

In [ ]:
df_equipped[df_equipped.hero=='Angelic Montmorancy']
# df_equipped[(df_equipped.hero.isin(['Faithless Lidica','Dizzy','Kayron','Tenebria']))]

#### Display and Sort Heroes by Stats

In [ ]:
st.GEAR_ENHANCE = 0  ## set to 0 for current stats.  
                     ##Setting to 15 would show hero stats if all gear is enhanced to 15 by increasing the min stat only
    
current_gear = df_hero_stats.apply(lambda row: fx.get_set_bonus(row, df_equipped), axis=1)
current_gear[['0','1','2','3','4','5']] = current_gear[['0','1','2','3','4','5']].fillna('z')
current_gear['Gear'] = str("['") + current_gear['0'] + str("','")+current_gear['1'] + str("','")+ \
            current_gear['2']+str("','")+ current_gear['3'] + str("','")+current_gear['4'] + \
            str("','")+current_gear['5'] + str("']")
current_gear = current_gear[current_gear.Complete == 1].copy().fillna(0).reset_index()
ee_df = pd.DataFrame(columns= ['Name','BonusStats','Atk','AtkP','HP','HPP','Def','DefP','Spd','CChance','CDmg','Eff','Res'])
ee_df['Name'] = current_gear['Name']
ee_df['BonusStats'] = current_gear['BonusStats']
ee_df = ee_df.fillna(0)
for index, row in ee_df.iterrows():
    char=row["Name"]
    for stat in np.unique(fx.gear_rating_lookup.stat_in.values):
        try: 
            val = row["BonusStats"][stat]
            ee_df.loc[index,stat] = val
        except: next
hero_inp_cols = ['Name','Level','Element','Role','Atk','HP','Speed','Def','Crit Rate','Crit Dmg','Effectiveness','Eff Resist','SC','EE']
hero_base_cols = ['Name','Lvl']
df = fx.get_combo_stats(current_gear[hero_base_cols].copy(), current_gear[hero_inp_cols], 
                        fx.mainst_sum(current_gear, df_equipped), 
                        fx.subst_sum(current_gear, df_equipped), 
                        fx.set_sum(current_gear), 
                        ee_df, 'all', target_stats['General'])

In [ ]:
df.sort_values(by = ['EHP'], ascending = False)

#### See Heroes with partially equipped gear

In [ ]:
for j in range(0,6):
    temp_df = df_hero_stats.merge(df_equipped[['hero','id']][df_equipped.Type == j], \
                        how = 'left', on = 'hero' )
    df_hero_stats[str(j)] = temp_df['id']

In [ ]:
partial_equip = df_equipped.groupby('hero').count().reset_index()
partial_equip[(partial_equip.id != 6)][['hero','id']]

#### Examine/Sort Gear

In [ ]:
df_items[
    (df_items.enhance < 15)  &  (df_items.level == 85) & \
    (df_items.set.isin(['Lifesteal'])) & (df_items.slot == 'Boots') ] \
    .sort_values(by = ['maxp'], ascending = False)

#### A method for finding lower rank gear items & removing them from json

Setting parameters or cutoffs for gear

In [ ]:
x1 = 60
x2 = 80
x3 = 10
x4 = df_items['current_eff'].quantile(q=0.3)

In [ ]:
low_items = df_items[(df_items.efficiency <= x1) & (df_items.max_eff <= x2) & \
                     (df_items.SPD <= x3) & (df_items.current_eff <= x4) & (df_items.SPD <= 8)].copy()

In [ ]:
print("number of items" , len(df_items))
print("number of unequipped items" , len(df_items[df_items.hero=='']))
print("number of items to delete" , len(low_items))
print("number of unequipped items to delete" , len(low_items[low_items.hero=='']))

In [ ]:
# view items
low_items

In [ ]:
#export/upload csv
low_items[low_items.hero==''][['id','efficiency','max_eff','set','slot','level','enhance']].to_csv('../outp/remove_list.csv')
remove = pd.read_csv('../outp/remove_list.csv')

##### deletes the gear from the json items and overwrites the upd_items.json (the same output from hero_optimizer)

In [ ]:
check1 = len(remove.id.values)
check2 = df_items[df_items.id.isin(remove.id.values)]['id'].count()

In [ ]:
if check1 == check2:
    print('pass', check1, check2)
    df_items = df_items.sort_values(by = ['hero','Type','efficiency','enhance'])
    export1 = df_items[~df_items.id.isin(remove.id.values)][['efficiency','hero','enhance','slot','level','set','rarity','mainStat','subStat1','subStat2','subStat3','subStat4','id','p_id','locked']].copy()
    export2 = export1.to_dict('records')
    with open('../outp/upd_items.json', 'w') as fp: json.dump(export2, fp)

In [ ]:
print( sc.df_items['id'].count() , export1['id'].count(), check1 )